In [1]:
import pandas as pd
import numpy as np


In [3]:
cpi = pd.read_csv("cpiinflation.csv")
income = pd.read_csv("incomedata.csv")
rent = pd.read_csv("rentdata.csv")


In [4]:
cpi.head()


,Statistic Label,Month,Commodity Group,UNIT,VALUE
0,Consumer Price Index (Base Dec 2023=100),1975 November,All items,Base Dec 2023=100,12.3
1,Consumer Price Index (Base Dec 2023=100),1975 November,Food and non-alcoholic beverages,Base Dec 2023=100,NaN
2,Consumer Price Index (Base Dec 2023=100),1975 November,Alcoholic beverages and tobacco,Base Dec 2023=100,NaN
3,Consumer Price Index (Base Dec 2023=100),1975 November,Clothing and footwear,Base Dec 2023=100,NaN
4,Consumer Price Index (Base Dec 2023=100),1975 November,"Housing, water, electricity, gas and other fuels",Base Dec 2023=100,NaN


In [5]:
income.head()


,Statistic Label,Quarter,Economic Sector 2 Digit NACE Rev 2,UNIT,VALUE
0,Earnings Per Week,2008Q1,Printing and reproduction of recorded media (18),Euro,758.56
1,Earnings Per Week,2008Q1,Basic pharmaceutical products and preparations...,Euro,1124.85
2,Earnings Per Week,2008Q1,Rubber and plastic products (22),Euro,605.61
3,Earnings Per Week,2008Q1,Other non-metallic mineral products (23),Euro,705.21
4,Earnings Per Week,2008Q1,"Computer, electronic and optical products (26)",Euro,911.89


In [6]:
rent.head()


,STATISTIC Label,Year,Number of Bedrooms,Property Type,Location,UNIT,VALUE
0,RTB Average Monthly Rent Report,2008,All bedrooms,All property types,Carlow,Euro,748.48
1,RTB Average Monthly Rent Report,2008,All bedrooms,All property types,Carlow Town,Euro,811.53
2,RTB Average Monthly Rent Report,2008,All bedrooms,All property types,"Graiguecullen, Carlow",Euro,711.35
3,RTB Average Monthly Rent Report,2008,All bedrooms,All property types,"Tullow, Carlow",Euro,720.04
4,RTB Average Monthly Rent Report,2008,All bedrooms,All property types,Cavan,Euro,571.72


In [7]:
cpi_clean = cpi[cpi["Commodity Group"] == "All items"].copy()


In [8]:
cpi_clean["Year"] = cpi_clean["Month"].str[:4].astype(int)


In [9]:
cpi_clean = cpi_clean[["Year", "VALUE"]]
cpi_clean.rename(columns={"VALUE": "CPI_Index"}, inplace=True)


In [10]:
cpi_clean.head()
cpi_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4808 entries, 0 to 62491
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       4808 non-null   int64  
 1   CPI_Index  4795 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 112.7 KB


In [12]:
cpi_yearly = (
    cpi_clean
    .groupby("Year", as_index=False)
    .mean()
)


In [13]:
cpi_yearly.head()


,Year,CPI_Index
0,1975,15.153846
1,1976,16.415116
2,1977,17.828125
3,1978,18.258333
4,1979,21.280208


In [28]:
rent_clean = rent.copy()

# standardise column names (remove weird spacing/case)
rent_clean.columns = rent_clean.columns.str.strip()



In [29]:
rent_clean["Year"] = pd.to_numeric(rent_clean["Year"], errors="coerce")
rent_clean = rent_clean.dropna(subset=["Year"])
rent_clean["Year"] = rent_clean["Year"].astype(int)


In [30]:
rent_clean["Number of Bedrooms"].value_counts().head(10)
rent_clean["Property Type"].value_counts().head(10)


Property Type
All property types     53074
Detached house         53074
Semi detached house    53074
Terrace house          53074
Apartment              53074
Other flats            53074
Name: count, dtype: int64

In [31]:
rent_filtered = rent_clean.copy()

# try filtering to "All bedrooms" if it exists
rent_filtered = rent_filtered[
    rent_filtered["Number of Bedrooms"].astype(str).str.lower().str.contains("all", na=False)
]

# try filtering to "All property types" if it exists
rent_filtered = rent_filtered[
    rent_filtered["Property Type"].astype(str).str.lower().str.contains("all", na=False)
]


In [32]:
rent_filtered.shape


(7582, 7)

In [33]:
rent_yearly = (
    rent_filtered
    .groupby(["Location", "Year"], as_index=False)["VALUE"]
    .mean()
)

rent_yearly.rename(columns={"VALUE": "Average_Rent"}, inplace=True)

rent_yearly.head()
rent_yearly.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Location      7582 non-null   object 
 1   Year          7582 non-null   int64  
 2   Average_Rent  7076 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 177.8+ KB


In [34]:
# ---- INCOME CLEANING ----

income_clean = income.copy()

# extract year from Quarter (e.g. 2008Q1)
income_clean["Year"] = income_clean["Quarter"].str[:4].astype(int)

# keep only required columns
income_clean = income_clean[["Year", "VALUE"]]
income_clean.rename(columns={"VALUE": "Average_Earnings"}, inplace=True)

# aggregate to yearly level
income_yearly = (
    income_clean
    .groupby("Year", as_index=False)
    .mean()
)

income_yearly.head()
income_yearly.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              18 non-null     int64  
 1   Average_Earnings  18 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 416.0 bytes


In [35]:
housing_df = rent_yearly.merge(
    income_yearly,
    on="Year",
    how="left"
)


In [36]:
housing_df.head()
housing_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Location          7582 non-null   object 
 1   Year              7582 non-null   int64  
 2   Average_Rent      7076 non-null   float64
 3   Average_Earnings  7582 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 237.1+ KB


In [37]:
housing_df = housing_df.merge(
    cpi_yearly,
    on="Year",
    how="left"
)


In [38]:
housing_df.head()
housing_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Location          7582 non-null   object 
 1   Year              7582 non-null   int64  
 2   Average_Rent      7076 non-null   float64
 3   Average_Earnings  7582 non-null   float64
 4   CPI_Index         7582 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 296.3+ KB


In [39]:
housing_df["Affordability_Ratio"] = (
    housing_df["Average_Rent"] / housing_df["Average_Earnings"]
)


In [40]:
housing_df["Real_Rent"] = (
    housing_df["Average_Rent"] / housing_df["CPI_Index"] * 100
)


In [42]:
housing_df["Rent_YoY_Growth"] = (
    housing_df
    .groupby("Location")["Average_Rent"]
    .pct_change(fill_method=None) * 100
)



In [43]:
housing_df["Income_YoY_Growth"] = (
    housing_df
    .groupby("Location")["Average_Earnings"]
    .pct_change(fill_method=None) * 100
)


In [44]:
housing_df.head()
housing_df.isna().sum()


Location                 0
Year                     0
Average_Rent           506
Average_Earnings         0
CPI_Index                0
Affordability_Ratio    506
Real_Rent              506
Rent_YoY_Growth        956
Income_YoY_Growth      446
dtype: int64

In [45]:
housing_df.to_csv("irish_housing_powerbi.csv", index=False)
